In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random

# 結果を保存するリスト
repositories = []

# User-Agentを設定してブラウザからのアクセスに見せる
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 処理するページ数を設定
pages_to_scrape = 100  # 最大100ページまで

for i in range(1, pages_to_scrape + 1):
    try:
        print(f"ページ {i}/{pages_to_scrape} を処理中...")
        
        url = f"https://github.com/search?q=google&type=repositories&p={i}"
        
        # リクエスト前に待機（GitHub側の負荷軽減）
        time.sleep(1)  
        
        response = requests.get(url, headers=headers)
        
        # レスポンスコードを確認
        if response.status_code != 200:
            print(f"エラー: ステータスコード {response.status_code}")
            if response.status_code == 429:
                print("レート制限に達しました。しばらく待ってから再開してください。")
            break
            
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # リポジトリのコンテナを探す
        repo_items = soup.select('div[data-testid="results-list"] > div')
        
        # 古い形式も試す（GitHubのHTMLが変更される可能性があるため）
        if not repo_items:
            repo_items = soup.select('li.repo-list-item')
        
        if not repo_items:
            print("リポジトリが見つかりません。GitHubのHTML構造が変更された可能性があります。")
            break
        
        print(f"{len(repo_items)}個のリポジトリが見つかりました")
        
        for repo in repo_items:
            try:
                # 1. リポジトリ名の取得（複数の可能なセレクタを試す）
                repo_name = "不明"
                
                # 新しいGitHubデザイン用
                repo_name_element = repo.select_one('a[data-testid="result-heading-title"]')
                if repo_name_element:
                    repo_name = repo_name_element.text.strip()
                else:
                    # 従来のデザイン用
                    repo_name_element = repo.select_one('a.v-align-middle')
                    if repo_name_element:
                        repo_name = repo_name_element.text.strip()
                
                time.sleep(1)  # 各要素の取得間に待機
                
                # 2. スター数の取得
                stars = "0"
                stars_element = repo.select_one('a[href*="stargazers"]')
                if stars_element:
                    stars = stars_element.text.strip().replace(',', '')
                
                time.sleep(1)  # 各要素の取得間に待機
                
                # 3. 言語の取得（複数の可能なセレクタを試す）
                language = "不明"
                
                # 新しいデザイン用
                language_elements = repo.select('span.Text-sc-17v1xeu-0')
                if language_elements:
                    for elem in language_elements:
                        prev_elem = elem.find_previous('span')
                        if prev_elem and prev_elem.get_text().strip() == "•":
                            language = elem.text.strip()
                            break
                
                # 従来のデザイン用
                if language == "不明":
                    language_element = repo.select_one('span[itemprop="programmingLanguage"]')
                    if language_element:
                        language = language_element.text.strip()
                
                time.sleep(1)  # 各要素の取得間に待機
                
                print(f"リポジトリ: {repo_name}, スター数: {stars}, 言語: {language}")
                
                repositories.append({
                    'repository': repo_name,
                    'stars': stars,
                    'language': language
                })
                
            except Exception as e:
                print(f"リポジトリの解析中にエラー: {e}")
        
        # ページ間の待機時間（GitHubのレート制限を回避）
        wait_time = random.uniform(2, 4)  # 2〜4秒のランダムな待機時間
        print(f"次のページを取得する前に {wait_time:.2f} 秒待機...")
        time.sleep(wait_time)
        
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        print("5分間待機してから続行します...")
        time.sleep(300)  # エラー発生時は5分待機
        continue

# 結果を保存
if repositories:
    df = pd.DataFrame(repositories)
    print(f"\n合計 {len(repositories)} 件のリポジトリが取得できました")
    
    # 重複を削除
    df = df.drop_duplicates(subset=['repository'])
    print(f"重複削除後: {len(df)} 件")
    
    # スター数を数値型に変換
    df['stars'] = pd.to_numeric(df['stars'], errors='coerce')
    
    # スター数でソート
    df = df.sort_values(by='stars', ascending=False)
    
    # 結果を表示
    print("\n上位10件:")
    print(df.head(10))
    
    # CSVに保存
    csv_filename = 'google_repositories.csv'
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"結果は {csv_filename} に保存されました")
else:
    print("リポジトリを取得できませんでした")

In [4]:
import sqlite3
import requests
from bs4 import BeautifulSoup
import time

# SQLiteデータベースを作成または接続
db_name = "google_repositories.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# テーブル作成（存在しない場合のみ作成）
cursor.execute('''
CREATE TABLE IF NOT EXISTS repositories (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    repository_name TEXT NOT NULL,
    primary_language TEXT NOT NULL,
    stars TEXT NOT NULL
)
''')
conn.commit()

# 90ページ分ループ
for i in range(1, 91):
    url = f"https://github.com/orgs/google/repositories?page={i}"
    res = requests.get(url)
    html_soup = BeautifulSoup(res.content, 'html.parser')

    # 各リポジトリ情報を取得
    tamesini = html_soup.find_all(class_="Text__StyledText-sc-17v1xeu-0 hWqAbU")  # リポジトリ名
    genngo = html_soup.find_all(class_="Text__StyledText-sc-17v1xeu-0 dNtsYW")  # プログラミング言語
    hosi = html_soup.find_all(class_="Box-sc-g0xbh4-0 XAPHx prc-Link-Link-85e08")  # スター数

    # データの挿入ループ
    for idx, (repo_name, language) in enumerate(zip(tamesini, genngo)):
        time.sleep(1)  # サーバー負荷軽減

        # スター数は 2, 6, 10...のインデックス
        star_index = 1 + idx * 4
        stars = hosi[star_index].get_text() if star_index < len(hosi) else "0"

        # データをデータベースに挿入
        cursor.execute('''
        INSERT INTO repositories (repository_name, primary_language, stars)
        VALUES (?, ?, ?)
        ''', (repo_name.get_text(), language.get_text(), stars))

        print(f"Saved: {repo_name.get_text()} | {language.get_text()} | {stars}")

    # ページごとにコミット
    conn.commit()

# データベース接続を閉じる
conn.close()
print("All data has been saved to the database.")

All data has been saved to the database.
